# Training the CNN Model for the 2D Spatial Components (Pytorch Version)

This notebook will help to demonstrate how to train the CNN Model used in CaImAn to evaluate the shape of (2p) spatial components using the Torch API.

The basic function for this is caiman.train.train_cnn_model_keras.cnn_model_pytorch(). It takes in the number of classes to build a CNN model. 

The other functions, caiman.train.helper.save_model_file() and caiman.train.helper.load_model_file() save and retrieve the model and weights of the model. 

Author: agiovanni, Manuel Paez

In [1]:
import numpy as np
import os
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torchvision.transforms.v2 as transforms

import caiman as cm
from caiman.paths import caiman_datadir
from caiman.train.helper import cnn_model_pytorch, get_batch_accuracy, load_model_pytorch, save_model_pytorch
from caiman.train.helper import train_test_split, train, validate 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

2025-02-26 17:00:12.726538: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 17:00:12.739250: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 17:00:12.742879: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 17:00:12.752257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740607212.767521 2483835 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740607212.77

False

## Initalizing the Parameters for the Model

In [2]:
batch_size = 128
num_classes = 2
epochs = 100
test_fraction = 0.25
augmentation = True #Fix this 
img_rows, img_cols = 50, 50 # input image dimensions
in_channels = 1

## Loading the Dataset of the Model 

In [3]:
with np.load('/mnt/ceph/data/neuro/caiman/labeling/k37_20160109_AM_150um_65mW_zoom2p2_00001_1-16/images/final_map/Yr_d1_512_d2_512_d3_1_order_C_frames_48000_.match_masks.npz') as ld:
# with np.load('/mnt/ceph/data/neuro/caiman/data_minions/ground_truth_components_curated_minions.npz') as ld: 
    all_masks_gt = torch.tensor(ld['all_masks_gt'], dtype=torch.float32) #define
    labels_gt = torch.tensor(ld['labels_gt_cur'], dtype=torch.long)

KeyError: 'all_masks_gt is not a file in the archive'

## Constructing the Training and Validation Set for the Model 

In [4]:
all_masks_gt = torch.reshape(all_masks_gt, (-1, in_channels, img_rows, img_cols))
dataset = TensorDataset(all_masks_gt, labels_gt) 

train_dataset, test_dataset = train_test_split(dataset, test_fraction)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_N = len(train_loader.dataset)
valid_loader = DataLoader(test_dataset, batch_size=batch_size)
valid_N = len(valid_loader.dataset)

NameError: name 'all_masks_gt' is not defined

## Build and Evaluate the Model 

In [7]:
model = cnn_model_pytorch(in_channels, num_classes)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train(model, train_loader, loss_function, optimizer, train_N, augment=None)
    validate(model, train_loader, loss_function, optimizer, valid_N, augment=None)

Epoch: 0
Train - Loss: 36.1255 Accuracy: 0.5869
Valid - Loss: 36.0095 Accuracy: 1.7607
Epoch: 1
Train - Loss: 35.9488 Accuracy: 0.5869
Valid - Loss: 35.9438 Accuracy: 1.7607
Epoch: 2
Train - Loss: 36.0044 Accuracy: 0.5869
Valid - Loss: 35.9368 Accuracy: 1.7607
Epoch: 3
Train - Loss: 35.9931 Accuracy: 0.5869
Valid - Loss: 35.9785 Accuracy: 1.7607
Epoch: 4


KeyboardInterrupt: 

## Save the Model and its weights

In [8]:
save_model_path = save_model_pytorch(model)

TypeError: save_model_pytorch() missing 1 required positional argument: 'name'

## Visualize the Results

In [ ]:
# predictions = model.predict(all_masks_gt, batch_size=32, verbose=1) fix this 
with torch.no_grad():
    predictions = model(all_masks_gt) 
    
A = torch.squeeze(all_masks_gt[torch.where(predictions[:, 0] >= 0.5)[0]]).numpy()
cm.movie(A).play(gain=3., magnification=5, fr=10)

tensor([[0.7271, 0.2729],
        [0.7409, 0.2591],
        [0.7388, 0.2612],
        ...,
        [0.7291, 0.2709],
        [0.7180, 0.2820],
        [0.6821, 0.3179]])
torch tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  .

## Retrieve the Model and its weights

In [ ]:
loaded_model = load_model_pytorch(save_model_path)
loaded_model.summary()

## Visualize Results 

In [ ]:
predictions = loaded_model.predict(all_masks_gt, batch_size=32, verbose=1)
cm.movie(np.squeeze(all_masks_gt[np.where(predictions[:, 0] >= 0.5)[0]])).play(
    gain=3., magnification=5, fr=10)